17.9 Maximizing house profit in a gamble and imputed probabilities

Primal problem:

minimize $t-p^Tx$,
subject to $Sx\leq t, 0\leq x\leq q$ 

here $S_{ij}=1$ iff outcome i is in participant j's set $S_j$

Lagrangian: $L(x,\lambda) = t-p^Tx+\lambda_1^T(Sx-t1)-\lambda_2^Tx+\lambda_3^T(x-q)$

Dual problem:

maximize $-\lambda_3^Tq$,
subject to $\lambda\geq 0, 1-\lambda_1^T1=0,\lambda_3=p+\lambda_2-S^T\lambda_1$

Primal problem for the expected profit:

minimize $(\pi^TS-p^T)x$,
subject to $0\leq x\leq q$

Lagrangian: $L(x,\lambda) = (\pi^TS-p^T)x-\lambda_2^Tx+\lambda_3^T(x-q)$

Dual problem for the expected profit:

maximize $-\lambda_3^Tq$,
subject to $\lambda\geq 0, \lambda_3=p+\lambda_2-S^T\pi$

So $\pi=\lambda_1^*$ is a distribution for which $x^*$ maximizes the expected house profit and the optimal values are the same.

In [1]:
import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt

In [58]:
n = 5  # 5 participants
m = 5  # 5 outcomes
p = np.array([.5,.6,.6,.6,.2])
q = np.array([10,5,5,20,10])
Si = [[1,2],[4],[1,4,5],[2,5],[3]]
S = np.zeros((m,n))
for i in range(n):
    for j in range(m):
        if j+1 in Si[i]:
            S[j][i] = 1
S

array([[1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0.],
       [0., 0., 1., 1., 0.]])

In [64]:
x = cp.Variable(n, nonneg=True)
t = cp.Variable()
objective = cp.Maximize(p @ x - t)
constraints = [
    x <= q,
    S @ x <= t]
prob = cp.Problem(objective, constraints)
result = prob.solve()
print('optimal sell:', x.value)
print('optimal worst-case house profit:', objective.value)
print('worst case house profit if all offers are accepted:', p @ q - np.max(S @ q))
pi = constraints[1].dual_value
print('imputed probabilities:', pi)
print('fair prices:', pi @ S)
x = cp.Variable(n, nonneg=True)
objective = cp.Maximize(p @ x - pi @ S @ x)
constraints = [x <= q]
prob = cp.Problem(objective, constraints)
result = prob.solve()
print('maximum expected profit for pi:', objective.value)

optimal sell: [ 5.  5.  5.  5. 10.]
optimal worst-case house profit: 3.499999996685668
worst case house profit if all offers are accepted: -5.0
imputed probabilities: [0.09699704 0.40300296 0.13155098 0.17145198 0.19699704]
fair prices: [0.5        0.17145198 0.46544607 0.6        0.13155098]
maximum expected profit for pi: 3.4999999902564696
